In [5]:
import numpy as np
from matplotlib import pyplot as plt

class GradientDescending():

    @staticmethod
    def grad(f, x, h=1e-5):
        return (f(x[:, np.newaxis] + h * np.eye(x.size)) - f(x[:, np.newaxis] - h * np.eye(x.size))) / (2 * h)

    def one_dimension_method(self, func, x, direction, alpha, eps, max_iterations):
        return x + direction * alpha

    def find_min(self, func, initial, alpha=0.4, eps=1e-3, max_iterations=1000):
        points = initial
        coords = initial
        for i in range(max_iterations):
            if i > 0:
                points = np.vstack((points, coords))
            direction = -self.grad(func, coords)
            next_coords = self.one_dimension_method(func, coords, direction, alpha, 1e-4, max_iterations)
            if np.max(np.abs(coords - next_coords)) < eps:
                return points
            coords = next_coords
        return points

class WolfGradientDescending(GradientDescending):
    c1 = 1e-4
    c2 = 0.999

    def check_first(self, func, x, direction, alpha, f_v, g_v):
        return func(x + alpha * direction) <= f_v + alpha * g_v * self.c1

    def check_second(self, func, x, direction, alpha, g_v):
        return np.dot(self.grad(func, x + alpha * direction), direction) >= self.c2 * g_v
    
    def find_alpha(self, func, x, direction, alpha):
        f_v = func(x)
        g_v = np.dot(self.grad(func, x), direction)
        l = 1e-4
        r = 100 * alpha
        ar = alpha
        while r - l > 1e-4:
            ar = (l + r) / 2
            if self.check_first(func, x, direction, ar, f_v, g_v):
                if self.check_second(func, x, direction, ar, g_v):
                    break
                else:
                    l = ar
            else:
                r = ar
        return ar

    def one_dimension_method(self, func, x, direction, alpha, eps, max_iterations):
        return x + direction * self.find_alpha(func, x, direction, alpha, func(x), np.dot(self.grad(func, x), direction))

class BFGS(WolfGradientDescending):

    def find_min(self, func, initial, alpha=1, eps=1e-3, max_iterations=1000):
        n = len(initial)
        I = np.eye(n)
        gr = self.grad(func, initial)
        hesian = I
        coords = initial
        points = initial
        for i in range(max_iterations):
            if i > 0:
                points = np.vstack((points, coords))
            if np.linalg.norm(gr) <= eps:
                return points
            direction = -np.dot(hesian, gr)
            alpha_k = self.find_alpha(func, coords, direction, alpha)
            coords_tmp = coords + alpha_k * direction
            coords_delta = coords_tmp - coords
            coords = coords_tmp

            gr_tmp = self.grad(func, coords + alpha_k * direction)
            gr_delta = gr_tmp - gr
            gr = gr_tmp

            ro = 1.0 / (np.dot(gr_delta, coords_delta) + 1e-8)
            A1 = I - ro * coords_delta[:, np.newaxis] * gr_delta[np.newaxis, :]
            A2 = I - ro * gr_delta[:, np.newaxis] * coords_delta[np.newaxis, :]
            hesian = np.dot(A1, np.dot(hesian, A2)) + (ro * coords_delta[:, np.newaxis] * coords_delta[np.newaxis, :])
        return points
    
class LBFGS(WolfGradientDescending):

    def compute_start_hesian(self, n, coords_delta, gr_delta):
        I = np.eye(n)
        return ((coords_delta[:, np.newaxis] * gr_delta[np.newaxis, :]) 
                / (gr_delta[:, np.newaxis] * gr_delta[np.newaxis, :] + 1e-8)) * I

    def compute_direction(self, coords_stored, grad_stored, gr, m, hesian):
        q = gr
        for i in range(m - 1, - 1, -1):
            ro_i = 1.0 / (np.dot(grad_stored[i], coords_stored[i]) + 1e-8)
            alpha_i = ro_i * np.transpose(coords_stored[i]) * q
            q = q - np.dot(alpha_i, grad_stored[i])
        r = np.dot(hesian, q)
        for i in range(m):
            ro_i = 1.0 / (np.dot(grad_stored[i], coords_stored[i]) + 1e-8)
            alpha_i = ro_i * np.transpose(coords_stored[i]) * q
            beta = ro_i * np.transpose(grad_stored[i]) * r
            r = r + coords_stored[i] * (alpha_i - beta)
        return -r


    def find_min(self, func, initial, alpha=1, eps=1e-3, m=40, max_iterations=1000):
        n = len(initial)
        I = np.eye(n)
        gr = self.grad(func, initial)
        hesian = I
        coords = initial
        points = initial
        coords_stored = []
        grad_stored = []
        for i in range(max_iterations):
            if i > 0:
                points = np.vstack((points, coords))
                hesian = self.compute_start_hesian(n, coords_stored[-1], grad_stored[-1])
            if np.linalg.norm(gr) <= eps:
                return points
            if i > m:
                direction = self.compute_direction(coords_stored, grad_stored, gr, m, hesian)
            else:
                direction = -np.dot(hesian, gr)
            alpha_k = self.find_alpha(func, coords, direction, alpha)
            coords_tmp = coords + alpha_k * direction
            coords_delta = coords_tmp - coords
            coords_stored.append(coords_delta)
            if i > m:
                coords_stored.pop(0)
            coords = coords_tmp
            gr_tmp = self.grad(func, coords + alpha_k * direction)
            gr_delta = gr_tmp - gr
            grad_stored.append(gr_delta)
            if i > m:
                grad_stored.pop(0)
            gr = gr_tmp
        return points


In [6]:
def f(x):
    return 2 * (x[0] - 10) ** 2 + 0.2 * (x[1] + 13) ** 2 + 3

def g(x):
    return (x[0] - 4) ** 2 + 100 * (x[1] + 5) ** 2

def h(x):
    return 4 * (x[0] - 1) ** 2 + 8 * (x[1] + 3) ** 2 - 4

def many_vars(x):
    n = len(x)
    result = 0
    for i in range(n):
        result += (i + 1) * (x[i] - (i + 1)) ** 2
    return result

initial = np.ones(2)
initial_many_vars = np.ones(100)

print("=====TESTING BFGS=====")
points = BFGS().find_min(f, initial)
print(f'Function: f, Iterations: {len(points)}, {points[-1]}')

points = BFGS().find_min(g, initial)
print(f'Function: g, Iterations: {len(points)}, {points[-1]}')

points = BFGS().find_min(h, initial)
print(f'Function: h, Iterations: {len(points)}, {points[-1]}')

points = BFGS().find_min(many_vars, initial_many_vars)
print(f'Function: many_vars, Iterations: {len(points)}, {points[-1]}')

print("=====TESTING LBFGS=====")
points = LBFGS().find_min(f, initial)
print(f'Function: f, Iterations: {len(points)}, {points[-1]}')

points = LBFGS().find_min(g, initial)
print(f'Function: g, Iterations: {len(points)}, {points[-1]}')

points = LBFGS().find_min(h, initial)
print(f'Function: h, Iterations: {len(points)}, {points[-1]}')

points = LBFGS().find_min(many_vars, initial_many_vars)
print(f'Function: many_vars, Iterations: {len(points)}, {points[-1]}')

=====TESTING BFGS=====
Function: f, Iterations: 26, [  9.9997128  -12.99949649]
Function: g, Iterations: 35, [ 4.0000272  -5.00000008]
Function: h, Iterations: 14, [ 1.        -3.0000059]
Function: many_vars, Iterations: 177, [  1.00000004   1.99999571   2.99999994   3.99999984   5.00000015
   5.99999692   7.00000006   7.99999868   9.00000188   9.99999781
  11.00000058  12.00000353  12.99999378  13.9999971   15.0000023
  16.00000008  17.00000085  17.99999867  18.99999988  20.00000089
  20.99999969  22.00000015  23.00000007  24.0000002   24.9999995
  26.00000043  26.99999994  28.0000002   28.99999979  30.00000081
  30.99999953  32.00000048  33.00000006  34.00000003  35.00000003
  35.99999991  37.00000038  37.99999924  39.0000007   39.99999954
  41.00000028  41.99999956  43.00000024  43.99999997  44.9999996
  46.00000065  46.99999936  48.00000053  48.99999929  50.00000056
  50.99999931  52.00000034  52.99999937  54.00000036  54.99999961
  55.99999995  56.9999999   58.00000003  58.9999997